# iprPy_fit testing



In [1]:
from pathlib import Path

from typing import Optional

import lammps

import atomman as am

import iprPy_fit

import tempfile

import numpy as np

from DataModelDict import DataModelDict as DM
import atomman as am
import iprPy

import iprPy_fit

import tempfile

import functools

from scipy.optimize import minimize

/home/lmh1/conda/envs/ipr/lib/python3.13/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


## LAMMPS options

In [2]:
lammps_exe = '/home/lmh1/LAMMPS/2022-06-23/src/lmp_serial'
lammps_exe_date = am.lammps.checkversion(lammps_exe)['date']
print(lammps_exe_date)

2022-06-23


In [3]:
lammps_lib = lammps.lammps(cmdargs=['-l', 'none', '-screen', 'none'])
lammps_lib_date = iprPy_fit.lammps.version_date(lammps_lib)
print(lammps_lib_date)

2025-07-22


## Potential definition

1. Creates a parambuilder object for a Tersoff modc potential and sets parameters. Alternatively, parameters could be loaded from an already existing tersoff.modc file.
2. Parameter file is saved inside a temporary directory.
3. Generates a potential object that is then used to build the LAMMPS commands for the potential.


In [4]:
# Define location for the parameter file
tempdir = tempfile.TemporaryDirectory()
filename = Path(tempdir.name, 'test.tersoff.modc')

# Build/load an initial potential
parambuilder = iprPy_fit.parambuilder.TersoffModC(symbols='Si')
params = {
    'Si_Si_Si_beta' :3.0,
    'Si_Si_Si_alpha' :2.257,
    'Si_Si_Si_h' :-0.3848,
    'Si_Si_Si_eta' :3.3044,
    'Si_Si_Si_beta_ters' :1,
    'Si_Si_Si_lambda2' :1.8753,
    'Si_Si_Si_B' :199.0625,
    'Si_Si_Si_R' :3.0709,
    'Si_Si_Si_D' :0.325,
    'Si_Si_Si_lambda1' :5.9134,
    'Si_Si_Si_A' :9422.1875,
    'Si_Si_Si_n' :4.7183,
    'Si_Si_Si_c1' :0.2795,
    'Si_Si_Si_c2' :403406.25,
    'Si_Si_Si_c3' :679093.75,
    'Si_Si_Si_c4' :4.4674,
    'Si_Si_Si_c5' :23.6047,
    'Si_Si_Si_c0' :-0.0039
}
parambuilder.update_parameter_values(**params)

# Saves the parameter file and generates an atomman potential object
potential = parambuilder.save_paramfile(filename, return_potential = True)

## System loading and transformations

- __ref_values__ is dict of target values
- __systems__ is list of atomic systems as atomman.System objects.
- __paramsets__ is list of dicts containing atomic and potential info that is likely more picklable.
- __scripts__ is list of pre-built LAMMPS scripts for evaluating each system.

In [5]:
# Initialize the ref_values dict
ref_values = {}
ref_keys = ['E_pot_total', 'E_pot_atom', 'P_xx', 'P_yy', 'P_zz']
for ref_key in ref_keys:
    ref_values[ref_key] = []

# Load system from the ref_system directory and extract ref_values
systems = []
for load_file in Path('ref_system').glob('*.json'):
    systems.append(am.load('system_model', load_file))
    model_values = DM(load_file)['ref_system']['reference_values']

    for ref_key in ref_keys:
        ref_values[ref_key].append(model_values[ref_key])

# Convert ref_values to numpy arrays
for ref_key in ref_keys:
    # Skip forces as each system may have a different # of atoms
    if ref_key == 'F': 
        continue
    ref_values[ref_key] = np.array(ref_values[ref_key])

# Convert to paramsets and pre-made LAMMPS scripts
paramsets = []
scripts_exe = []
scripts_lib = []
for system in systems:
    paramsets.append(iprPy_fit.lammps.dump_lammps_dynamic_parameters(system, potential=potential, return_pair_info=True))
    scripts_exe.append(iprPy_fit.lammps.build_script(potential, system, lammps_exe_date))
    scripts_lib.append(iprPy_fit.lammps.build_script(potential, system, lammps_lib_date))

print(len(systems), 'atomic systems found')

83 atomic systems found


## Evaluation tests

### From executable and scripts

In [14]:
results = iprPy_fit.evaluate.evaluate(lammps_exe, scripts=scripts_exe)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -2.79715166e+01, -3.66423108e+01,
        -2.79715166e+01, -3.66423108e+01, -2.79715166e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [9]:
%%timeit
results = iprPy_fit.evaluate.evaluate(lammps_exe, scripts=scripts_exe)

7.61 s ± 92.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### From executable and systems

In [15]:
results = iprPy_fit.evaluate.evaluate(lammps_exe, systems=systems, potential=potential)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -3.66423108e+01, -3.66423108e+01,
        -3.66423108e+01, -3.66423108e+01, -3.66423108e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [12]:
%%timeit
results = iprPy_fit.evaluate.evaluate(lammps_exe, systems=systems, potential=potential)

7.85 s ± 66.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### From library and scripts

In [13]:
results = iprPy_fit.evaluate.evaluate(lammps_lib, scripts=scripts_lib)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -3.66423108e+01, -3.66423108e+01,
        -3.66423108e+01, -3.66423108e+01, -3.66423108e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [19]:
%%timeit
results = iprPy_fit.evaluate.evaluate(lammps_lib, scripts=scripts_lib)

3.17 s ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### From library and systems

In [16]:
results = iprPy_fit.evaluate.evaluate(lammps_lib, systems=systems, potential=potential)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -3.66423108e+01, -3.66423108e+01,
        -3.66423108e+01, -3.66423108e+01, -3.66423108e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [20]:
%%timeit
results = iprPy_fit.evaluate.evaluate(lammps_lib, systems=systems, potential=potential)

882 ms ± 8.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### From library and paramsets

In [17]:
results = iprPy_fit.evaluate.evaluate(lammps_lib, paramsets=paramsets)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -3.66423108e+01, -3.66423108e+01,
        -3.66423108e+01, -3.66423108e+01, -3.66423108e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [21]:
%%timeit
results = iprPy_fit.evaluate.evaluate(lammps_lib, paramsets=paramsets)

844 ms ± 7.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Same but auto-create new lammps object

In [18]:
results = iprPy_fit.evaluate.evaluate(None, paramsets=paramsets)
results

{'E_pot_total': array([-3.66422574e+01, -3.66423642e+01, -3.66422574e+01, -3.66423642e+01,
        -3.66422574e+01, -3.66423642e+01, -3.66423108e+01, -3.66423108e+01,
        -3.66423108e+01, -3.66423108e+01, -3.66423108e+01, -3.66423108e+01,
        -1.83208549e+01, -3.87987471e+00, -2.07360147e+02, -7.56158744e+00,
        -2.43841399e+02, -1.15240111e+01, -1.57021811e+01, -5.88614298e+02,
        -6.10605277e+01, -3.66423108e+01, -3.61795030e+01, -8.34468138e+01,
        -3.66412958e+01, -3.00731002e+01, -1.56826877e+01, -6.12690593e+01,
        -7.73798169e+01, -6.15167938e+01, -1.64888487e+01, -7.15299496e+01,
        -2.60913869e+01, -7.13510828e+01, -6.14899472e+02, -7.86232782e+00,
        -3.62069621e+01, -1.03894998e+02, -6.18781856e+02, -4.24772981e+00,
        -1.59438265e+01, -7.00593249e+01, -2.54858539e+01, -3.10377202e+01,
        -3.29611906e+01, -4.61375372e+00, -3.84444804e+00, -3.19822646e+00,
        -2.65782053e+00, -2.20722790e+00, -1.83224880e+00, -1.52059464e+0

In [22]:
%%timeit
results = iprPy_fit.evaluate.evaluate(None, paramsets=paramsets)

847 ms ± 8.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Minimize testing

In [6]:
# Fitting weights
weights = {
    'E_pot_total': 1.0,
    'E_pot_atom': 0.0,
    'P_xx': 1000.0,
    'P_yy': 1000.0,
    'P_zz': 1000.0,
}

# This is the set of parameters to fit with bounds
param_bounds = {
    'Si_Si_Si_alpha' : (0.00, 5.00),
    'Si_Si_Si_h' : (-0.60, -0.20),
    'Si_Si_Si_eta' : (1.00, 5.00),
    'Si_Si_Si_lambda2' : (0.00, 10.00),
    'Si_Si_Si_B' : (0.00, 10000.00),
    'Si_Si_Si_R' : (2.50, 3.50),
    'Si_Si_Si_D' : (0.25, 0.50),
    'Si_Si_Si_lambda1' : (0.00, 10.00),
    'Si_Si_Si_A' : (0.00, 10000.00),
    'Si_Si_Si_n' : (1.00, 5.00),
    'Si_Si_Si_c1' : (0.00, 1.00),
    'Si_Si_Si_c2' : (0.00, 1000000.00),
    'Si_Si_Si_c3' : (0.00, 1000000.00),
    'Si_Si_Si_c4' : (0.00, 5.00),
    'Si_Si_Si_c5' : (0.00, 50.00),
    'Si_Si_Si_c0' : (-0.01, 0.01)
}

In [8]:
iprPy_fit.minimize.minimize(
    parambuilder,
    filename,
    param_bounds,
    ref_values,
    weights,
    
    lmp = lammps_lib,
    paramsets = paramsets,
    min_method='Nelder-Mead',
    min_options={'maxiter':100, 'adaptive':True})

Initial error is 22141208.311783895
Final error is 5736830.9163812045


{'Si_Si_Si_alpha': 2.2927214911894556,
 'Si_Si_Si_h': -0.38602443195947256,
 'Si_Si_Si_eta': 3.3916047429801295,
 'Si_Si_Si_lambda2': 1.8669123120683149,
 'Si_Si_Si_B': 198.96762841606107,
 'Si_Si_Si_R': 3.367665259147547,
 'Si_Si_Si_D': 0.32533083732301205,
 'Si_Si_Si_lambda1': 5.7095059155366785,
 'Si_Si_Si_A': 9514.367960944008,
 'Si_Si_Si_n': 4.4497940058546,
 'Si_Si_Si_c1': 0.2814990014569999,
 'Si_Si_Si_c2': 411012.0472001805,
 'Si_Si_Si_c3': 666524.2437689696,
 'Si_Si_Si_c4': 4.510372375636377,
 'Si_Si_Si_c5': 23.66937747767878,
 'Si_Si_Si_c0': -0.004006241818848739}

In [9]:
iprPy_fit.minimize.minimize(
    parambuilder,
    filename,
    param_bounds,
    ref_values,
    weights,
    
    lmp = lammps_lib,
    paramsets = paramsets,
    min_method='Nelder-Mead',
    min_options={'maxiter':100, 'adaptive':True})

Initial error is 6070327.255511911
Final error is 4234963.188562262


{'Si_Si_Si_alpha': 2.5404634461012634,
 'Si_Si_Si_h': -0.363609358312349,
 'Si_Si_Si_eta': 3.3645654048461817,
 'Si_Si_Si_lambda2': 1.868346542424447,
 'Si_Si_Si_B': 198.75956432186018,
 'Si_Si_Si_R': 3.351547148261032,
 'Si_Si_Si_D': 0.32559127285576217,
 'Si_Si_Si_lambda1': 6.074570862192461,
 'Si_Si_Si_A': 9347.229793527576,
 'Si_Si_Si_n': 4.380010368695751,
 'Si_Si_Si_c1': 0.28680216194053293,
 'Si_Si_Si_c2': 412904.83385455015,
 'Si_Si_Si_c3': 665439.1800191515,
 'Si_Si_Si_c4': 4.474462095271358,
 'Si_Si_Si_c5': 23.407480718531332,
 'Si_Si_Si_c0': -0.003989298155030531}